In [1]:
import warnings
import itertools
import pandas
import math
import sys
import os
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# Configure your choices here: choose detector, profile, and file (dataset) to show in the visualizations.
# This is the only file which you should need to edit as a user. 

detector='htmcoredistal' #choose detector here, ie. 'numenta', 'htmcore'
detector_profile = "standard" # choose from: "reward_low_FP_rate", "reward_low_FN_rate", "standard"
detector_summary_row = "artificialWithAnomaly/art_daily_flatmiddle.csv" # select detector summary file row(data file name) 
# ...from the list produced below, or see `./results/`

In [3]:
#Reading summary
detector_summary_file = "../results/"+str(detector)+"/"+str(detector)+"_"+str(detector_profile)+"_scores.csv"

summaryDataFrame = pandas.read_csv(detector_summary_file,index_col="File") 
assert 'Score' in summaryDataFrame.columns, "'Score' column missing in data file"
    
summaryDataFrame = summaryDataFrame.sort_values(by=['Score'],ascending=False)
print("Printing scores for selected profile (NaN represents total score):")
print(summaryDataFrame.loc[:, 'Score'])

Printing scores for selected profile (NaN represents total score):
File
artificialWithAnomaly/art_daily_flatmiddle.csv   -0.78779
NaN                                              -0.78779
Name: Score, dtype: float64


In [4]:
# Add detector result file(a single file for specific data file)
path_,file_ = detector_summary_row.split("/")
result_file = "../results/"+str(detector)+"/"+str(path_)+"/"+str(detector)+"_"+str(file_)
assert os.path.isfile(result_file), "No such file: "+str(result_file)
assert set(['Threshold','TP','TN','FP','FN','Total_Count']).issubset(summaryDataFrame.columns), "Missing columns in file "+str(detector_summary_file)

TP,TN,FP,FN= None,None,None,None
assert summaryDataFrame["Threshold"][detector_summary_row].size==1, "There is multiple records in summary file for one datafile!" 
threshold = summaryDataFrame["Threshold"][detector_summary_row]
TP = summaryDataFrame["TP"][detector_summary_row]
TN = summaryDataFrame["TN"][detector_summary_row]
FP = summaryDataFrame["FP"][detector_summary_row]
FN = summaryDataFrame["FN"][detector_summary_row]
total_Count = summaryDataFrame["Total_Count"][detector_summary_row]
standard_score = np.array(summaryDataFrame['Score'][detector_summary_row])
        
print("For result file : " + result_file)
print("True Positive (Detected anomalies) : " + str(TP))
print("True Negative (Detected non anomalies) : " + str(TN))
print("False Positive (False alarms) : " + str(FP))
print("False Negative (Anomaly not detected) : " + str(FN))
print("Total data points : " + str(total_Count))
print(detector_profile+" score : "+str(np.sum(standard_score)))

For result file : ../results/htmcoredistal/artificialWithAnomaly/htmcoredistal_art_daily_flatmiddle.csv
True Positive (Detected anomalies) : 15
True Negative (Detected non anomalies) : 3124
False Positive (False alarms) : 16
False Negative (Anomaly not detected) : 273
Total data points : 3428
standard score : -0.7877895440495589


In [5]:
#Ploting Result (Note if the plot is not visible please close the tab shutdown notebook and restart)
#Reading results file 

dataframe = pandas.read_csv(result_file)
assert set(['timestamp','value','anomaly_score','label']).issubset(dataframe.columns), "Missing columns in file:"+str(result_file)+" columns:"+str(dataframe.columns)    

x = np.array(dataframe['timestamp'])
value = np.array(dataframe['value'])
anomaly_score = np.array(dataframe['anomaly_score'])
anomaly_label = np.array(dataframe['label'])
standard_score = np.array(dataframe['S(t)_standard'])

# Plot values, anomaly score and label scaled to values
value_max = np.max(value)
trace_value = {"x": x,
             "y": value,
             "mode": 'lines',
             "name": 'Value'}

trace_anomaly_score = {"x": x,
                  "y": anomaly_score*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly score'}

trace_anomaly_label = {"x": x,
                  "y": anomaly_label*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly window'}
trace_threshold = {"x": x,
                  "y": np.ones(len(x))*threshold*value_max,
                  "mode": 'lines',
                  "name": 'Anomaly threshold'}


traces = [trace_value,trace_anomaly_score,trace_threshold,trace_anomaly_label]
layout = dict(title = "Scalled anomaly score with value : "+result_file,
                  xaxis = dict(title = 'X'),
                  yaxis = dict(title = 'Value')
                 )


fig = dict(data=traces, layout=layout)
iplot(fig)

In [6]:
#plot, anomalys score, label, and result from benchmark
trace_anomaly_score = {"x": x,
                  "y": anomaly_score,
                  "mode": 'lines',
                  "name": 'Anomaly score'}

trace_anomaly_label = {"x": x,
                  "y": anomaly_label,
                  "mode": 'lines',
                  "name": 'Anomaly window'}

trace_threshold = {"x": x,
                  "y": np.ones(len(x))*threshold,
                  "mode": 'lines',
                  "name": 'Anomaly threshold'}

trace_standard_score = {"x": x,
                  "y": standard_score,
                  "mode": 'lines',
                  "name": 'standard_score'}

traces = [trace_anomaly_score,trace_threshold,trace_anomaly_label,trace_standard_score]
layout = dict(title = "Anomaly score : "+result_file,
                  xaxis = dict(title = 'X'),
                  yaxis = dict(title = 'Value', range=[-0.15, 1.1]) #range = anomaly score range [0,1] + some padding
                 )

fig = dict(data=traces, layout=layout)
iplot(fig)

In [7]:
## This is a different file for plotting data 

# Add your data file to plot
data_file = "../data/artificialWithAnomaly/art_daily_flatmiddle.csv"

# Plot Data (Note if the plot is not visible please close the tab shutdown notebook and restart)
assert os.path.isfile(data_file), "No such file:"+str(data_file)
dataframe = pandas.read_csv(data_file)

assert set(['timestamp','value']).issubset(dataframe.columns), "Missing columns in file:"+str(data_file)+" , cols:"+str(dataframe.columns)

x = np.array(dataframe['timestamp'])
y = np.array(dataframe['value'])

mean = np.mean(y)

trace = {"x": x,
             "y": y,
             "mode": 'lines',
             "name": 'Value'}
trace_mean = {"x": x,
                  "y": np.ones(len(x))*mean,
                  "mode": 'lines',
                  "name": 'Mean'}
traces = [trace,trace_mean]
layout = dict(title = "Data plot : "+data_file,
                  xaxis = dict(title = 'X'),
                  yaxis = dict(title = 'Value')
                 )
fig = dict(data=traces, layout=layout)
iplot(fig)